# Make Master Flat Frames
### MSP, Week of 2.15.2018

In [1]:
# python 2/3 compatibility
from __future__ import print_function

# numerical python
import numpy as np

# file management tools
import glob
import os

# good module for timing tests
import time

# plotting stuff
import matplotlib.pyplot as plt
%matplotlib inline

# ability to read/write fits files
from astropy.io import fits

# fancy image combination technique
from astropy.stats import sigma_clipping


### Step 0: Make Defintions

These are unchanged (though better documented) from previous versions. In order to avoid lots of boilerplate code, I have moved the basic definitions to a new python script, HDI_io.py.

In [2]:
# make the definitions for reading in images accessible in this notebook.
from HDI_io import *

print(read_image.__doc__)
print(arrange_quadrants.__doc__)


    read_image
    ----------
    routine to read in an image and either return
    
    mosaic==True
        single mosaicked, non-overscan frame
        
    mosaic==False
        dictionary of numbered quadrants for overscan operations
        
        (can later be combined with arrange_quadrants)
        
        
    inputs
    ----------
    infile         : (string) filename to be read in
    mosaic         : (boolean, default=True) if True, returns a single data frame,
                     if False, returns a dictionary of the four quadrants
    
    outputs
    ----------
    data_quad      : (dictionary or array) if dictionary, keys are [0,1,2,3], each 2056x2048
                     corresponding to each quadrant. if array, single 4122x4096 frame
    
    overscan_quad  : (dictionary) keys are [0,1,2,3], each 2056x2048, corresponding to each quadrant


    dependents
    ----------
    arrange_quadrants : definition to place quadrants in the correct configuration, below

  

A little definition to check that the filter is set correctly.

In [3]:

def verify_hdi_filter(infile,des_filter,verbose=False):
    '''
    verify_filter
    ---------------
    check that the filter we think is being used IS being used
    
    inputs
    ---------
    infile      : (string) filename of image to check
    des_filter  : (string) common filter name (see filter_dict below)
    verbose     : (boolean, default=False) if True, print reason for failure
    
    returns
    ---------
    0,1,2       : (int code) 0=filter matches; 1=filter does not match; 2=one wheel slot not empty
    
    '''
    
    # specify the filter dictionary; hardcoded because it will not change
    filter_dict = {'V':'102',\
               'R':'103',\
               'I':'104',\
               'Ha':'200',\
               'Hao':'204',\
              'empty':['105','106','107','205','206','207']}
    
    # get the header
    phdr = fits.getheader(infile,0)
    
    # check that a wheel slot is empty
    if ((phdr['FILTER1'] not in filter_dict['empty']) & (phdr['FILTER2'] not in filter_dict['empty']) ):
        
        if verbose:
            print('verify_filter: failed with multiple filters.')
            
        return 2
    
    # check that filter matches the desired wheel position
    elif  (( phdr['FILTER1'] == filter_dict[des_filter]) | ( phdr['FILTER2'] == filter_dict[des_filter]) ):
        return 0
    
    # default to failure mode
    else:
        if verbose:
            print('verify_filter: failed with non-matching filter.')
            
        return 1
    
    
    

### Flat-making definitions

There are two ways to construct the flat:
1. normalizing by the median of the entire image
2. normalizing each quadrant by it's own median

here is the routine that uses the median of the entire image.

In [4]:
def make_flat_single_median(flat_files,verbose=False,sigma_clip=False,verify=False,filt=''):
    '''
    make_flat_single_median
    -----------------------
    routine to take an array of flat files and 
    1. read in to array
    2. subtract overscan from each quadrant
    3. normalize by exposure time
    4. take the median of the array of images
    5. normalize resultant image by *median of entire frame*
    
    inputs
    ------------
    flat_files    : (list) input list of images
    verbose       : (boolean, default=False) print filter checking (>0) and timing tests (>1)
    sigma_clip    : (boolean, default=False) if True, use sigma clipping; else use median
    verify        : (boolean, default=False) if True, performs verification of filter wheel position
    filt          : (string, default=None) if verify=True, will perform filter comparison
    
    outputs
    ------------
    masterflat    : (matrix, 4112x4096) single frame
    
    '''
    if ( (verify)& (filt == '') ):
        raise ValueError('verify=True requires filt to be set.')
    
    t1 = time.time()

    # initialize an array for the files
    flat = np.zeros([len(flat_files),4112,4096])

    # loop through the images
    for imgnum,img in enumerate(flat_files):
        hdr = fits.getheader(img,0)
        
        if verify:
            code = verify_hdi_filter(img,filt.strip('d'),verbose=verbose)
            
            if code:
                print('make_flat_single_median: skipping bad file {}'.format(img.split('/')[-1]))
                continue
        
        # mosaic=False means that these come out as dictionaries of the extensions
        flat_quads,ovrscn = read_image(img,mosaic=False)
        
        for ext in flat_quads.keys(): 
            # overscan subtract the quadrants
            flat_quads[ext] = flat_quads[ext] - np.median(ovrscn[ext])
        
        # now arrange the quadrants in the single median version
        arranged_flat = arrange_quadrants(flat_quads)
            
        # also don't forget some twilight flats have different exposure times
        flat[imgnum] = arranged_flat/hdr['EXPTIME']
            
    t2 = time.time()
    if (verbose > 1): print('time elapsed to read:',np.round(t2-t1,2),' seconds')    
        
    if sigma_clip:
        clipped_arr = sigma_clipping.sigma_clip(flat,sigma=3,axis=0)
        median_clipped_arr = np.ma.median(clipped_arr,axis=0)
        masterflat = median_clipped_arr.filled(0.)
        
    else:
        # median and normalize the single-value frame
        masterflat = np.median(flat,axis=0)
        
    masterflat /= np.median(masterflat)
             
    return masterflat





# Flats.

In [5]:
# set up global directory structure

indir = '/Volumes/AST337/A341_2018/ObservingRunData/'

# shadow directory for reductions
reducedir = '/Volumes/AST337/reduced/'




## Use Lists

These are the flat files from each night to use.

This is not the most graceful solution: it relies on specifying the data.

Please check your flats and the logs and let me know if there are differences! (or flaws in certain images)

In [6]:

# list of files
flat_list = {}

night = '20180113'
cnight = '8130'
ext = 'f00.fits'
# night 1
flat_list[night] = {}
flat_list[night]['Vd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(739,744)]
flat_list[night]['Rd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(746,749)]

# the julian day ticked over on this set
cnight = '8131'
flat_list[night]['Rd'].extend([indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(1,3)])
flat_list[night]['Id'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(4,9)]
flat_list[night]['Had'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(11,16)]
flat_list[night]['Haod'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(18,22)]


# night 2
night = '20180114'
cnight = '8132'
ext = 'f00.fits'
flat_list[night] = {}

# these are dome flats
flat_list[night]['Vd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(10,17)]
flat_list[night]['Rd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(19,26)]
flat_list[night]['Id'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(27,34)]

# these are twilight flats
flat_list[night]['V'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(576,581)]
flat_list[night]['R'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(582,588)]
flat_list[night]['I'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(588,596)]
flat_list[night]['Ha'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(597,602)]
flat_list[night]['Hao'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(607,612)]


# night 3
night = '20180115'
cnight = '8133'
ext = 'f00.fits'
flat_list[night] = {}
# these are dome flats
flat_list[night]['Vd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(684,689)]
flat_list[night]['Rd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(689,694)]
flat_list[night]['Id'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(699,704)]
flat_list[night]['Had'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(704,709)]
flat_list[night]['Haod'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(710,715)]

# these are twilight flats
flat_list[night]['V'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(3,9)]
flat_list[night]['R'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(15,19)]
flat_list[night]['I'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(9,15)]


# night 4
night = '20180116'
cnight = '8134'
ext = 'f00.fits.gz'
flat_list[night] = {}
# these are dome flats
flat_list[night]['Vd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(679,684)]
flat_list[night]['Rd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(685,690)]
flat_list[night]['Id'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(690,695)]
flat_list[night]['Had'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(668,673)]
flat_list[night]['Haod'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(674,679)]


# night 5
night = '20180117'
cnight = '8135'
ext = 'f00.fits'
flat_list[night] = {}
flat_list[night]['Vd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(2,14)]
flat_list[night]['Rd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(16,28)]
flat_list[night]['Id'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(28,40)]
flat_list[night]['Had'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(42,53)]
flat_list[night]['Haod'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(56,68)]


# night 6
night = '20180118'
cnight = '8136'
ext = 'f00.fits'
flat_list[night] = {}
flat_list[night]['Vd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(2,7)]
flat_list[night]['Rd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(8,13)]
flat_list[night]['Id'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(14,18)]

# twilight flats
flat_list[night]['V'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(19,23)]
flat_list[night]['R'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(23,29)]
flat_list[night]['I'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(30,36)]
flat_list[night]['Ha'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(804,809)]
flat_list[night]['Hao'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(812,815)]
flat_list[night]['Hao'].extend([indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(818,820)])


# night 7
night = '20180119'
cnight = '8137'
ext = 'f00.fits.gz'
flat_list[night] = {}
flat_list[night]['Vd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(2,7)]
flat_list[night]['Rd'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(7,12)]
flat_list[night]['Id'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(12,17)]
flat_list[night]['Had'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(17,23)]
flat_list[night]['Haod'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(24,29)]

# twilight flats
flat_list[night]['V'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(44,47)]
flat_list[night]['V'].extend([indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(819,824)])
flat_list[night]['R'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(38,41)]
flat_list[night]['R'].extend([indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(834,839)])
flat_list[night]['I'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(29,34)]
flat_list[night]['I'].extend([indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(846,853)])
flat_list[night]['Ha'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(853,858)]
flat_list[night]['Hao'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(858,865)]



# night 8
night = '20180120'
cnight = '8138'
ext = 'f00.fits.gz'
flat_list[night] = {}
flat_list[night]['V'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(25,35)]
flat_list[night]['R'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(35,41)]
flat_list[night]['I'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(41,47)]
flat_list[night]['Ha'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(12,17)]
flat_list[night]['Hao'] = [indir+night+'/c'+cnight+'t0{0:03d}'.format(x)+ext for x in range(17,23)]




## Define the Directories

In [7]:
# nights are split into different folders
datadirs = ['20180113','20180114','20180115','20180116','20180117','20180118','20180119','20180120']

# which filters. appended 'd' means dome flat, otherwise they are twilights
filters = ['Vd','Rd','Id','Had','Haod','V','R','I','Ha','Hao']



## Run the Loops

Loops for
1. Night
2. Filter

In [8]:

tt = time.time()

# loop through nights of data
for night in datadirs:
    print(night)
    
    tn = time.time()
    
    for filter in filters:
        
        # check which sets of flats are actually defined for a given night
        if filter not in flat_list[night].keys():
            continue
        
        flat_files = flat_list[night][filter]
    
        t0 = time.time()

        # this is the entire flat maker, given a set of flat files
        # note that I have sigma clipping on here--this is SLOW!
        masterflat = make_flat_single_median(flat_files,verbose=1,sigma_clip=True,verify=True,filt=filter)
        
        phdr = fits.getheader(flat_files[0],0)
        fits.writeto(reducedir+night+'/masterflat_{}.fits'.format(filter),masterflat,phdr,overwrite=True)
        
        # be obsessive about timing
        t3 = time.time()
        print('time elapsed to make flat:',np.round(t3-t0,2),' seconds')
        
    # be more obsessive about timing
    print('time elapsed for {} flats:'.format(night),np.round((time.time()-tn)/60.,1),' minutes',end='\n\n')

# one more timing test
print('time elapsed for ALL flats:',np.round((time.time()-tt)/60.,1),' minutes')

20180113
time elapsed to make flat: 70.47  seconds
time elapsed to make flat: 68.95  seconds
time elapsed to make flat: 66.32  seconds
time elapsed to make flat: 68.62  seconds
time elapsed to make flat: 62.06  seconds
time elapsed for 20180113 flats: 5.6  minutes
20180114
time elapsed to make flat: 98.24  seconds
time elapsed to make flat: 94.66  seconds
time elapsed to make flat: 100.62  seconds
time elapsed to make flat: 68.95  seconds
time elapsed to make flat: 79.83  seconds
time elapsed to make flat: 111.52  seconds
time elapsed to make flat: 65.21  seconds
time elapsed to make flat: 66.2  seconds
time elapsed for 20180114 flats: 11.4  minutes
20180115
time elapsed to make flat: 71.42  seconds
time elapsed to make flat: 69.73  seconds
time elapsed to make flat: 82.03  seconds
time elapsed to make flat: 84.26  seconds
time elapsed to make flat: 77.32  seconds
time elapsed to make flat: 88.6  seconds
time elapsed to make flat: 66.81  seconds
time elapsed to make flat: 86.48  second